![terrainbento logo](../images/terrainbento_logo.png)


# Introduction to boundary conditions in terrainbento.


## Overview
This tutorial shows example usage of the terrainbento boundary handlers. For comprehensive information about all options and defaults, refer to the [documentation](http://terrainbento.readthedocs.io/en/latest/). 

## Prerequisites
This tutorial assumes you have at least skimmed the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/) and worked through the [Introduction to terrainbento](http://localhost:8888/notebooks/example_usage/Introduction_to_terrainbento.ipynb) tutorial.

### terrainbento boundary handlers
terrainbento includes five boundary handlers designed to make it easier to treat different model run boundary conditions. Four boundary handlers modify the model grid in order to change the base level the model sees. The final one calculates how changes in precipitation distribution statistics change the value of erodibility by water. Hyperlinks in the list below go to the documentation for each of the boundary condition handlers. 

1. [`CaptureNodeBaselevelHandler`](http://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_condition_handlers.capture_node_baselevel_handler.html) implements external drainage capture. 
2. [`SingleNodeBaselevelHandler`](http://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_condition_handlers.single_node_baselevel_handler.html) modifies the elevation of one model grid node, intended to be the outlet of a modeled watershed. 
3. [`NotCoreNodeBaselevelHandler`](http://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_condition_handlers.not_core_node_baselevel_handler.html) either increments all the core nodes, or all the not-core nodes up or down. 
4. [`GenericFuncBaselevelHandler`](http://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_condition_handlers.generic_function_baselevel_handler.html) is a generic boundary condition handler that modifies the model grid based on a user specified function of the model grid and model time. 
5. [`PrecipChanger`](http://terrainbento.readthedocs.io/en/latest/source/terrainbento.boundary_condition_handlers.precip_changer.html) modifies precipitation distribution parameters (in **St** models) or erodibility by water (all other models). 

If you have additional questions related to using the boundary handlers or your research requires additonal tools to handle boundary conditions, please let us know by making an [Issue on GitHub](https://github.com/TerrainBento/terrainbento/issues). 

In the `SingleNodeBaselevelHandler` and the `NotCoreNodeBaselevelHandler`, rate of baselevel fall at a single node or at all not-core model grid nodes can be specified as a constant rate or a time-elevation history. These and other options are described in the documentation. Note that a model instance can have more than one boundary handler at a time.

The swiss-army knife of boundary condition handling is the `GenericFuncBaselevelHandler` so we will focus on it today. 

### Example Usage

To begin, we will import the required python modules. 

In [ ]:
import numpy as np
np.random.seed(42)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import holoviews as hv
hv.notebook_extension('matplotlib')

from terrainbento import Basic

Rather than taking a constant baselevel fall rate, the `GenericFuncBoundaryHandler` takes a function. This function is expected to accept two arguments --- the model grid and the elapsed model integration time --- and return an array of size number-of-model-grid-nodes that represents the spatially variable rate of boundary lowering or core-node uplift. 

For our example we will create a model grid initially at ~1000 m elevation at all grid nodes, then we will progressively drop the model boundary elevations. We will vary the spatial and temporal pattern of boundary elevations such that the boundaries will drop more rapidly at the beginning of the model run than at the end and the boundaries will drop more on the bottom of the model grid domain than on the top.  

If you are not familiar with user defined python functions, consider reviewing [this tutorial](https://www.datacamp.com/community/tutorials/functions-python-tutorial#udf). 

Thus our function will look as follows:

In [ ]:
def dropping_boundary_condition_1(grid, t):
    f = 0.007
    dzdt = -1. * (2e5 - t) / 2e5 * f * (
        (grid.y_of_node.max() - grid.y_of_node) / grid.y_of_node.max())
    return dzdt

Importantly, note that this function returns the *rate* at which the boundary will drop, *not* the elevation of the boundary through time. 

Next we construct the parameter dictionary we need to initialize the terrainbento model. For this example we will just use the **Basic** model. 

In order to specify that we want to use the `GenericFuncBaselevelHandler` we provide it as a value to the parameter `BoundaryHandlers`. We can provide the parameters the baselevel handler needs directly in the parameter dictionary, or we can create a new sub-dictionary, as is done below. 

In [ ]:
import numpy as np
np.random.seed(42)
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 2e5
    },

    # Create the Grid.
    "grid": {
        "RasterModelGrid": [(25, 40), {
            "xy_spacing": 40
        }, {
            "fields": {
                "node": {
                    "topographic__elevation": {
                        "random": [{
                            "where": "CORE_NODE"
                        }],
                        "constant": [{
                            "value": 1000.
                        }]
                    }
                }
            }
        }]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "GenericFuncBaselevelHandler": {
            "function": dropping_boundary_condition_1
        }
    },

    # Parameters that control output.
    "output_interval": 5e3,
    "save_first_timestep": True,
    "output_prefix": "model_basic_output_intro_bc",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.0001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.01,
}

Next we create a model instance, run it, create an xarray dataset of the model output, and convert it to the holoviews format. 

In [ ]:
basic = Basic.from_dict(basic_params)
basic.run()
ds = basic.to_xarray_dataset(time_unit='years', space_unit='meters')
hvds_topo = hv.Dataset(ds.topographic__elevation)

Finally we create an image of the topography with a slider bar. 

In [ ]:
%opts Image style(interpolation='bilinear', cmap='viridis') plot[colorbar=True]
topo = hvds_topo.to(hv.Image, ['x', 'y'], label='Rate Decreases')
topo

### Contrasting with a slightly different boundary condition

If we wanted a different pattern, we would just need to change the function. For example, if we wanted to compare a similar boundary condition in which the rate of boundary lowering increased through time instead of decreasing through time our function would look as follows:

In [ ]:
def dropping_boundary_condition_2(grid, t):
    f = 0.007
    dzdt = -1. * (t) / 2e5 * f * (
        (grid.y_of_node.max() - grid.y_of_node) / grid.y_of_node.max())
    return dzdt

Next we will make a new model that is exactly the same as the other model but uses the new function and a different output file name. 

In [ ]:
second_model_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 2e5
    },

    # Create the Grid.
    "grid": {
        "RasterModelGrid": [(25, 40), {
            "xy_spacing": 40
        }, {
            "fields": {
                "node": {
                    "topographic__elevation": {
                        "random": [{
                            "where": "CORE_NODE"
                        }],
                        "constant": [{
                            "value": 1000.
                        }]
                    }
                }
            }
        }]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "GenericFuncBaselevelHandler": {
            "function": dropping_boundary_condition_2
        }
    },

    # Parameters that control output.
    "output_interval": 5e3,
    "save_first_timestep": True,
    "output_prefix": "model_basic_new_bc",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.0001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.01,
}

basic2 = Basic.from_dict(second_model_params)
basic2.run()
ds2 = basic2.to_xarray_dataset(time_unit='years', space_unit='meters')
hvds_topo2 = hv.Dataset(ds2.topographic__elevation)

Finally we plot both models together to compare them. 

In [ ]:
%opts Image style(interpolation='bilinear', cmap='viridis') plot[colorbar=True]
topo2 = hvds_topo2.to(hv.Image, ['x', 'y'], label='Rate Increases')
topo + topo2

As you can see, the landscapes created by the **Basic** model with the two slightly different boundary conditions are different. One thing to think about is what sort of geologic settings might create each of these two alternative boundary conditions and how you could quantitatively compare these two output landscapes. 

Finally we remove the xarray datasets from and use the model function `remove_output_netcdfs` to remove the files created by running the model. 

In [ ]:
del topo, hvds_topo, topo2, hvds_topo2

ds.close()
ds2.close()
basic.remove_output_netcdfs()
basic2.remove_output_netcdfs()

## Next Steps

- We recommend you review the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/).

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](Introduction_to_terrainbento.ipynb) 
    
    2) **This Notebook**: [Introduction to boundary conditions in terrainbento](introduction_to_boundary_conditions.ipynb)
    
    3) [Introduction to output writers in terrainbento](introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](../coupled_process_elements/model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](../coupled_process_elements/model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](../coupled_process_elements/model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](../coupled_process_elements/model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](../coupled_process_elements/model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values